In [ ]:

#=++> we no longer need this code as we are installing the package via wheel file in the job environment
#import os
#import sys
#current_dir = os.getcwd()
#project_root = os.path.abspath(os.path.join(current_dir, "../../../"))
##print(project_root, current_dir)
#if project_root not in sys.path:
#    sys.path.append(project_root)


In [ ]:
#----> we remove the src because we added src/citibike  and src/utils to the packages in pyproject.toml file
#from src.citibike.citibike_utils import get_trip_duration_mins
#from src.utils.datetime_utils import timestamp_to_date_col
#from pyspark.sql.functions import create_map, lit


from citibike.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [2]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")



c:\Users\TBR44\Desktop\databricks\dab\dab_project\.venv_dbc\Lib\site-packages\databricks\sdk\_widgets\__init__.py:71: UserWarning: 
To use databricks widgets interactively in your notebook, please install databricks sdk using:
	pip install 'databricks-sdk[notebook]'
Falling back to default_value_only implementation for databricks widgets.
  warnings.warn(


KeyError: 'pipeline_id'

In [3]:
df = spark.read.table(f"{catalog}.01_bronze.jc_citibike")

NameError: name 'catalog' is not defined

In [5]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [6]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [10]:
df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

NameError: name 'pipeline_id' is not defined

In [ ]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )

In [8]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.02_silver.jc_citibike")